In [2]:
import pandas as pd
from pathlib import Path
import uuid

In [8]:
fincausal_task1_train_path = Path('/media/sarthak/HDD/data_science/fnp_resources/data/task1/iteration_2/train.csv')
semeval_all_data_path = Path('/media/sarthak/HDD/data_science/fnp_resources/data/extra/semeval.csv')

In [9]:
semeval_all_data_df = pd.read_csv(semeval_all_data_path)
semeval_all_data_df.head()

,sentence,relation,causal
0,The system as described above has its greatest...,Component-Whole,0
1,The child was carefully wrapped and bound into...,Other,0
2,The author of a keygen uses a disassembler to ...,Instrument-Agency,0
3,A misty ridge uprises from the surge.,Other,0
4,The student association is the voice of the un...,Member-Collection,0


In [10]:
semeval_all_data_df.shape

(10717, 3)

In [22]:
fincausal_task1_train_df = pd.read_csv(fincausal_task1_train_path)
fincausal_task1_train_df.head()

,Index,Text,Gold,unique_id
0,81.00009,Look at which clients are delivering increasi...,0,849c40bc-8fce-433b-8dc8-6af727480679
1,357.00082,From foundation modules that introduce asset ...,0,694a6824-613c-4978-9e17-a45b77e2b485
2,471.00003,The law firm Kessler Topaz Meltzer & Check LL...,0,661e6bf7-ffbf-4581-acff-8c246ecac51a
3,166.00001,Upcoming events Learn about the requirements...,0,48a694c5-9372-48d2-b2cb-df9461c713fe
4,48.00012,"The Midwest, where Buffett is from, has a his...",0,a363e188-0393-433b-a86c-94d268eb473c


In [23]:
fincausal_task1_train_df.shape

(15575, 4)

In [24]:
# what's the distribution of label?
causal_distribution = fincausal_task1_train_df.Gold.value_counts() / len(fincausal_task1_train_df)
causal_distribution

0    0.927897
1    0.072103
Name: Gold, dtype: float64

In [25]:
# how many causal can I take from semeval?
semeval_noncausal = semeval_all_data_df.loc[semeval_all_data_df.causal == 0]
semeval_causal = semeval_all_data_df.loc[semeval_all_data_df.causal == 1]

semeval_causal_resampled = semeval_causal.sample(n=int((causal_distribution[1]*100*len(semeval_noncausal))/(100-(causal_distribution[1]*100))))

semeval_resampled = pd.concat([semeval_noncausal, semeval_causal_resampled])
semeval_resampled.causal.value_counts() / len(semeval_resampled)

0    0.927929
1    0.072071
Name: causal, dtype: float64

In [26]:
# combine both the datasets
semval_resampled_formatted_df = pd.DataFrame({
    'Index': None,
    'Text': semeval_resampled.sentence,
    'Gold': semeval_resampled.causal,
    'unique_id': [str(uuid.uuid4()) for _ in range(len(semeval_resampled))]
})
semval_resampled_formatted_df.head()

,Index,Text,Gold,unique_id
0,None,The system as described above has its greatest...,0,3581f615-136e-4e1c-9249-86a7c438632a
1,None,The child was carefully wrapped and bound into...,0,6143cd7e-f404-44da-9b5b-663de269e93e
2,None,The author of a keygen uses a disassembler to ...,0,01a1f60b-fa7a-4991-b4db-4841f985c2b1
3,None,A misty ridge uprises from the surge.,0,b559eaf3-1391-4d33-94eb-ade7d6f172f4
4,None,The student association is the voice of the un...,0,66b0e9de-9fc7-4740-94bc-f34664d954c2


In [27]:
semval_resampled_formatted_df.Gold.value_counts() / len(semval_resampled_formatted_df)

0    0.927929
1    0.072071
Name: Gold, dtype: float64

In [28]:
fincausal_task1_train_combined_semeval = pd.concat([fincausal_task1_train_df, semval_resampled_formatted_df])
fincausal_task1_train_combined_semeval.shape

(25690, 4)

In [29]:
fincausal_task1_train_combined_semeval.Gold.value_counts() / len(fincausal_task1_train_combined_semeval)

0    0.92791
1    0.07209
Name: Gold, dtype: float64

In [30]:
fincausal_task1_train_combined_semeval.to_csv('/media/sarthak/HDD/data_science/fnp_resources/data/task1/iteration_2/train_with_semeval.csv', index=False)